In [1]:
import pandas as pd
import numpy as np

In [ ]:
path = 'dns_oct-jan.csv'
data = pd.read_csv(path)
data['day'] = pd.to_datetime(data.day)
data = data[['shieldid', 'day', 'dns_qname', 'dns_qdomainname', 'num_clients', 'num_responses']]
data

In [ ]:
pivot_table = data.pivot_table(values = 'shieldid', index = ['dns_qdomainname', 'day'], columns = None, aggfunc = 'nunique') 
pivot_table.reset_index(inplace = True)
counts = pivot_table.groupby('dns_qdomainname')['day'].nunique()
filtered_df = pivot_table.loc[pivot_table['dns_qdomainname'].isin(counts[counts <= 7].index)] #change the number "7" for how many days you want this domain_name to show up in the dataset
anomalies = filtered_df[filtered_df.shieldid > 3].set_index('day')
data.loc[:,'anomaly'] = 0
data.loc[data.dns_qdomainname.isin(anomalies.dns_qdomainname.to_list()),'anomaly'] = 1
anomalies['shieldid_count'] = anomalies.shieldid
anomalies = anomalies.drop('shieldid', axis = 1)
anomalies = anomalies.reset_index()
pivot_table['shieldid_count'] = pivot_table.shieldid
pivot_table = pivot_table.drop('shieldid', axis = 1)
data2 = pd.merge(data, pivot_table, on= ['day', 'dns_qdomainname'], how = 'left')